<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/yolov5_gradCAM_corneAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5_GradCAM_CorneAI**

https://github.com/pooya-mohammadi/yolov5-gradcam

##**Setup YOLOv5**

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd /content
!pip uninstall deep_utils -y
!pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git --q
!pip install torch --q
!pip install torchvision --q
!pip install -U opencv-python --q
print("[INFO] To use new installed version of opencv, the session should be restarted!!!!")

!git clone https://github.com/pooya-mohammadi/yolov5-gradcam

/content
  Preparing metadata (setup.py) ... done
[INFO] To use new installed version of opencv, the session should be restarted!!!!
Cloning into 'yolov5-gradcam'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 134 (delta 29), reused 36 (delta 13), pack-reused 68 (from 1)
Receiving objects: 100% (134/134), 6.95 MiB | 25.59 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [3]:
import os
os.chdir('/content/yolov5-gradcam')

model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/3.jpg"

In [ ]:
# !python main.py --model-path $model_path --img-path $img_path --output-dir out

#**Top3 analysis**

In [ ]:
import os
img_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
img_dir = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit"
print(len(os.listdir(img_dir)))

234


In [ ]:
import gc
gc.collect()


26

In [ ]:
#################
#################

import numpy as np
from deep_utils.utils.box_utils.boxes import Box
import torch
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
import cv2
import time
import torchvision
import torch.nn as nn
from utils.metrics import box_iou

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")
        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()
        # fetch the names
        if names is None:
            print('[INFO] fetching names from coco file')
            self.names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                          'traffic light',
                          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                          'cow',
                          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase',
                          'frisbee',
                          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                          'surfboard',
                          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
                          'apple',
                          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                          'couch',
                          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                          'keyboard', 'cell phone',
                          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                          'teddy bear',
                          'hair drier', 'toothbrush']
        else:
            self.names = names

        # preventing cold start
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        """Runs Non-Maximum Suppression (NMS) on inference and logits results

        Returns:
             list of detections, on (n,6) tensor per image [xyxy, conf, cls] and pruned input logits (n, number-classes)
        """

        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            # Apply constraints
            # x[((x[..., 2:4] < min_wh) | (x[..., 2:4] > max_wh)).any(1), 4] = 0  # width-height
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            # Cat apriori labels if autolabelling
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            # If none remain process next image
            if not x.shape[0]:
                continue

            # Compute conf
            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            # log_ *= x[:, 4:5]
            # Box (center x, center y, width, height) to (x1, y1, x2, y2)
            box = xywh2xyxy(x[:, :4])

            # Detections matrix nx6 (xyxy, conf, cls)
            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                # log_ = x[:, 5:]
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            # Filter by class
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            # Check shape
            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            elif n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence

            # Batched NMS
            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:  # limit detections
                i = i[:max_det]
            if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
                # update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
                iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
                weights = iou * scores[None]  # box weights
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
                if redundant:
                    i = i[iou.sum(1) > 1]  # require redundancy

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break  # time limit exceeded

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):

        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                for *xyxy, conf, cls in det:
                    # xyxyの値を修正
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(img_size, xyxy[2])
                    xyxy[3] = min(img_size, xyxy[3])

                    bbox = Box.box2box(xyxy,
                                       in_source=Box.BoxSource.Torch,
                                       to_source=Box.BoxSource.Numpy,
                                       return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img





#**GradCAM**

In [ ]:
"""
GradCAM通常バージョン
"""

import os
import time
import numpy as np
# from models.gradcam import YOLOV5GradCAM
#from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

# 以下の関数は変更なし
def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

# YOLOV5GradCAMクラスを更新
class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []  # クラス名を格納するリストを初期化

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))


    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()
                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        return alpha.view(b, k, 1, 1)

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)
        positive_gradients = F.relu(score.exp() * gradients)
        weights = (positive_gradients * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)
        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)

        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)

        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)

        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)

        return eigen_cam.unsqueeze(1)

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, method):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])

    saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size, method=method)

    tic = time.time()
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    print("total time:", round(time.time() - tic, 4))

    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]  # convert to bgr
    images = [result]

    for i in range(len(masks)):
        res_img = result.copy()
        for j, mask in enumerate(masks[i]):
            bbox = boxes[0][j]
            res_img, _ = get_res_img(bbox, [mask], res_img)
            res_img = put_text_box(bbox, cls_names[i], res_img)
        images.append(res_img)

    final_image = concat_images(images)
    img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
    output_path = f'{output_dir}/{img_name}'
    os.makedirs(output_dir, exist_ok=True)
    print(f'[INFO] Saving the final image at {output_path}')
    cv2_imshow(final_image)

def folder_main(folder_path, method):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    #####ファイル名を数字でソート################
    file_list = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))[200:234]
    #############################################

    for item in file_list:
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image: {img_basename}")
        img = cv2.imread(img_path)

        # 元の画像の縦横比を維持しながらリサイズ
        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        img = cv2.resize(img, (new_w, new_h))

        # パディングを追加して640×640pxにする
        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(img[..., ::-1])

        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size, method=method)

        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"boxes: {boxes}")

        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr
        images = [result]

        for i in range(len(masks)):
            res_img = result.copy()
            for j, mask in enumerate(masks[i]):
                bbox = boxes[0][j]
                res_img, _ = get_res_img(bbox, [mask], res_img)
                res_img = put_text_box(bbox, cls_names[i], res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')

        #cv2.imwrite(output_path, final_image)
        cv2_imshow(final_image)

        # メモリ解放
        del saliency_method, masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ
    #target_layer = 'model_17_cv3_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_20_cv3_conv'  # GradCAM を適用するレイヤー
    target_layer = 'model_23_cv3_conv'  # GradCAM を適用するレイヤー →Best
    #target_layer = 'model_23_cv3_act'  # GradCAM を適用するレイヤー  →デフォルト
    ##target_layer = 'model_23_m_0_cv2_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_23_m_0_cv2_act'  # GradCAM を適用するレイヤー
    #target_layer = 'model_24_m_1'  # GradCAM を適用するレイヤー
    ##target_layer = 'model_24_m_2'  # GradCAM を適用するレイヤー
    #img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    #output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_slit'  # 出力ディレクトリ
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho'  # 出力ディレクトリ


    # GradCAMとGradCAM++の両方を実行
    #for method in ['gradcam', 'gradcampp', 'eigencam']:
    for method in ['gradcam', 'gradcampp']:
    #for method in ['gradcampp']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            folder_main(img_path, method)
        else:
            main(img_path, method)


In [ ]:
"""
GradCAM
広く見るバージョン
"""

import os
import time
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def apply_heatmap_to_entire_image(mask, image):
    mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    n_heatmap = (heatmap / 255).astype(np.float32)
    image = image / 255
    res_img = cv2.addWeighted(image, 0.7, n_heatmap, 0.3, 0)
    return res_img

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()
                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        eigenvalues, eigenvectors = torch.linalg.eigh(cov)

        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)

        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)

        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)

        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)

        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def folder_main(folder_path, method):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    file_list = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))[210:211] ###########画像を指定

    for item in file_list:
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image: {img_basename}")
        img = cv2.imread(img_path)

        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        img = cv2.resize(img, (new_w, new_h))

        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(img[..., ::-1])

        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size, method=method)

        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"boxes: {boxes}")

        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr
        images = [result]

        for i in range(len(masks)):
            res_img = result.copy()
            res_img = apply_heatmap_to_entire_image(masks[i], res_img)
            for j, bbox in enumerate(boxes[0]):
                res_img = put_text_box(bbox, cls_names[i], res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')

        cv2_imshow(final_image)

        del saliency_method, masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ
    target_layer = 'model_23_cv3_conv'  # GradCAM を適用するレイヤー
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho'  # 出力ディレクトリ

    for method in ['gradcam', 'gradcampp']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            folder_main(img_path, method)
        else:
            main(img_path, method)

In [ ]:
import os
import time
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def apply_heatmap_to_entire_image(mask, image):
    mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    n_heatmap = (heatmap / 255).astype(np.float32)
    image = image / 255
    res_img = cv2.addWeighted(image, 0.7, n_heatmap, 0.3, 0)
    return res_img

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def print_model_structure(model):
    """
    モデルの構造を表示する関数
    """
    print("Model structure:")
    for i, (name, module) in enumerate(model.model.named_modules()):
        if not isinstance(module, torch.nn.Sequential) and not isinstance(module, torch.nn.ModuleList):
            print(f"{i}: {name} - {module.__class__.__name__}")

def find_yolo_layer(model, layer_index):
    """
    YOLOv5モデルの後ろから指定されたインデックスのレイヤーを返す
    """
    module_list = list(model.model.modules())
    module_list = [m for m in module_list if not isinstance(m, torch.nn.Sequential) and not isinstance(m, torch.nn.ModuleList)]
    target_layer = module_list[-layer_index]
    return target_layer, f"{target_layer.__class__.__name__} (index: -{layer_index})"


class YOLOV5GradCAM:
    def __init__(self, model, layer_index, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer, layer_name = find_yolo_layer(self.model, layer_index)
        self.layer_name = layer_name
        print(f"Selected layer: {self.layer_name}")
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()
                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        eigenvalues, eigenvectors = torch.linalg.eigh(cov)

        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)

        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)

        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)

        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)

        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def folder_main(folder_path, method, saliency_method):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    print(f"Using layer: {saliency_method.layer_name}")

    file_list = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))[200:234]

    for item in file_list:
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image: {img_basename}")
        img = cv2.imread(img_path)

        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        img = cv2.resize(img, (new_w, new_h))

        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(img[..., ::-1])

        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"boxes: {boxes}")

        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr
        images = [result]

        for i in range(len(masks)):
            res_img = result.copy()
            res_img = apply_heatmap_to_entire_image(masks[i], res_img)
            for j, bbox in enumerate(boxes[0]):
                res_img = put_text_box(bbox, cls_names[i], res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')

        cv2_imshow(final_image)

        del masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ
    layer_index = 1  # モデルの後ろから1番目のレイヤーを選択
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho'

    model = YOLOV5TorchObjectDetector(model_path, device, img_size=(img_size, img_size), names=names)

    for method in ['gradcam', 'gradcampp']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            saliency_method = YOLOV5GradCAM(model=model, layer_index=layer_index, img_size=(img_size, img_size), method=method)
            folder_main(img_path, method, saliency_method)
        else:
            main(img_path, method)

###**EigenCAM**

In [ ]:
import os
import time
import numpy as np
# from models.gradcam import YOLOV5GradCAM
#from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()
                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        # GradCAMの重み計算
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        # GradCAM++の重み計算
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)

        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)

        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)

        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)

        # Resize to match the input image size
        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)

        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, method):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])

    saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size, method=method)

    tic = time.time()
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    print("total time:", round(time.time() - tic, 4))

    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]  # convert to bgr
    images = [result]

    for i in range(len(masks)):
        res_img = result.copy()
        for j, mask in enumerate(masks[i]):
            bbox = boxes[0][j]
            res_img, _ = get_res_img(bbox, [mask], res_img)
            res_img = put_text_box(bbox, cls_names[i], res_img)
        images.append(res_img)

    final_image = concat_images(images)
    img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
    output_path = f'{output_dir}/{img_name}'
    os.makedirs(output_dir, exist_ok=True)
    print(f'[INFO] Saving the final image at {output_path}')
    cv2_imshow(final_image)

def folder_main(folder_path, method):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    #####ファイル名を数字でソート################
    file_list = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))[6:15]
    #############################################

    for item in file_list:
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image: {img_basename}")
        img = cv2.imread(img_path)

        # 元の画像の縦横比を維持しながらリサイズ
        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        img = cv2.resize(img, (new_w, new_h))

        # パディングを追加して640×640pxにする
        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(img[..., ::-1])

        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size, method=method)

        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"boxes: {boxes}")

        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr
        images = [result]

        for i in range(len(masks)):
            res_img = result.copy()
            for j, mask in enumerate(masks[i]):
                bbox = boxes[0][j]
                res_img, _ = get_res_img(bbox, [mask], res_img)
                res_img = put_text_box(bbox, cls_names[i], res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')

        #cv2.imwrite(output_path, final_image)
        cv2_imshow(final_image)

        # メモリ解放
        del saliency_method, masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ
    #target_layer = 'model_17_cv3_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_20_cv3_conv'  # GradCAM を適用するレイヤー
    target_layer = 'model_23_cv3_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_23_cv3_act'  # GradCAM を適用するレイヤー  →デフォルト
    #target_layer = 'model_23_m_0_cv2_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_23_m_0_cv2_act'  # GradCAM を適用するレイヤー
    #target_layer = 'model_24_m_0'  # GradCAM を適用するレイヤー
    #target_layer = 'model_24_m_1'  # GradCAM を適用するレイヤー
    #target_layer = 'model_24_m_2'  # GradCAM を適用するレイヤー
    #img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    #output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_slit'  # 出力ディレクトリ
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho'  # 出力ディレクトリ


    # GradCAMとGradCAM++の両方を実行
    #for method in ['gradcam', 'gradcampp', 'eigencam']:
    #for method in ['gradcam', 'gradcampp']:
    for method in ['eigencam']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            folder_main(img_path, method)
        else:
            main(img_path, method)


In [ ]:
"""
EigenCAM
広く可視化するバージョン
"""

import os
import time
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def apply_heatmap_to_entire_image(mask, image):
    mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    n_heatmap = (heatmap / 255).astype(np.float32)
    image = image / 255
    res_img = cv2.addWeighted(image, 0.7, n_heatmap, 0.3, 0)
    return res_img

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        res_img = apply_heatmap_to_entire_image(mask, res_img)
    return res_img, None

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()
                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        eigenvalues, eigenvectors = torch.linalg.eigh(cov)

        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)

        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)

        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)

        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)

        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def folder_main(folder_path, method):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    file_list = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))[6:15]

    for item in file_list:
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image: {img_basename}")
        img = cv2.imread(img_path)

        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        img = cv2.resize(img, (new_w, new_h))

        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(img[..., ::-1])

        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size, method=method)

        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"boxes: {boxes}")

        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr
        images = [result]

        for i in range(len(masks)):
            res_img = result.copy()
            res_img, _ = get_res_img(None, [masks[i]], res_img)
            for j, bbox in enumerate(boxes[0]):
                res_img = put_text_box(bbox, cls_names[i], res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')

        cv2_imshow(final_image)

        del saliency_method, masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ
    target_layer = 'model_20_cv3_conv'  # GradCAM を適用するレイヤー
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho'  # 出力ディレクトリ

    for method in ['eigencam']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            folder_main(img_path, method)
        else:
            main(img_path, method)

In [ ]:
model = YOLOV5TorchObjectDetector(model_path, device, img_size=(img_size, img_size), names=names)
print(model)

In [ ]:
import sys
sys.exit()

#**LayerごとTop1の可視化**

In [ ]:
import os
import time
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F
import torchvision
import torch.nn as nn
from utils.general import xywh2xyxy
from utils.datasets import letterbox
from utils.metrics import box_iou
from models.experimental import attempt_load

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")
        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()
        if names is None:
            print('[INFO] fetching names from coco file')
            self.names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                          'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                          'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
                          'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
                          'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',
                          'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
                          'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                          'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
                          'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
        else:
            self.names = names

        # preventing cold start
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            elif n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence

            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:  # limit detections
                i = i[:max_det]
            if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
                iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
                weights = iou * scores[None]  # box weights
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
                if redundant:
                    i = i[iou.sum(1) > 1]  # require redundancy

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break  # time limit exceeded

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                for *xyxy, conf, cls in det:
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(img_size, xyxy[2])
                    xyxy[3] = min(img_size, xyxy[3])

                    bbox = Box.box2box(xyxy,
                                       in_source=Box.BoxSource.Torch,
                                       to_source=Box.BoxSource.Numpy,
                                       return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img): ##マージしているレイヤー名
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()

                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)
        # Compute covariance matrix
        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)
        # Compute eigenvectors
        eigenvalues, eigenvectors = torch.linalg.eigh(cov)
        # Select the eigenvector corresponding to the largest eigenvalue
        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)
        # Compute EigenCAM
        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)
        # Normalize
        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)
        # Resize to match the input image size
        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)
        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, method):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])

    #target_layers = ['model_17_cv3_conv', 'model_20_cv3_conv', 'model_23_cv3_conv']
    target_layers = ['model_24_m_0', 'model_24_m_1', 'model_24_m_2'] ##
    images = [img]  # 元画像を追加

    for layer_name in target_layers:
        saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name, img_size=input_size, method=method)
        tic = time.time()
        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"{layer_name} time:", round(time.time() - tic, 4))
        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr

        res_img = result.copy()
        if len(masks) > 0:  # masksが空でない場合のみ処理を行う
            for j, mask in enumerate(masks[0]):
                bbox = boxes[0][j]
                res_img, _ = get_res_img(bbox, [mask], res_img)
                res_img = put_text_box(bbox, f"{cls_names[0]} - {method} - {layer_name}", res_img)
        images.append(res_img)

    final_image = concat_images(images)
    img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
    output_path = f'{output_dir}/{img_name}'
    os.makedirs(output_dir, exist_ok=True)
    print(f'[INFO] Saving the final image at {output_path}')
    #cv2_imshow(final_image)
    cv2.imwrite(output_path, final_image)


def folder_main(folder_path, method, file_list):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    for index, item in enumerate(file_list, start=1):
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image {index}: {img_basename}")
        img = cv2.imread(img_path)

        # 元の画像の縦横比を維持しながらリサイズ
        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        resized_img = cv2.resize(img, (new_w, new_h))

        # パディングを追加して640×640pxにする
        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        padded_img = cv2.copyMakeBorder(resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(padded_img[..., ::-1])

        #target_layers = ['model_17_cv3_conv', 'model_20_cv3_conv', 'model_23_cv3_conv'] ##
        target_layers = ['model_17_cv3_act', 'model_20_cv3_act', 'model_23_cv3_act'] ##
        ##target_layers = ['model_17_m_0_cv2_conv', 'model_20_m_0_cv2_conv', 'model_23_m_0_cv2_conv']
        #target_layers = ['model_24_m_0', 'model_24_m_1', 'model_24_m_2'] ##
        images = [padded_img]  # 元画像を追加

        for layer_name in target_layers:
            saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name, img_size=input_size, method=method)
            masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
            print(f"boxes: {boxes}")

            result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
            result = result[..., ::-1]  # convert to bgr

            res_img = result.copy()
            if len(masks) > 0:  # masksが空でない場合のみ処理を行う
                for j, mask in enumerate(masks[0]):
                    bbox = boxes[0][j]
                    res_img, _ = get_res_img(bbox, [mask], res_img)
                    res_img = put_text_box(bbox, f"{cls_names[0]}-{'layer'+layer_name.replace('model_', '')}", res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')
        cv2_imshow(final_image) ##画像を表示
        #cv2.imwrite(output_path, final_image) ##画像を保存


        # メモリ解放
        del saliency_method, masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ

    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
    #img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_slit_layer24'  # 出力ディレクトリ
    #output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho_layer24'  # 出力ディレクトリ

    # ファイル名を数字でソート
    if os.path.isdir(img_path):
        file_list = sorted(os.listdir(img_path), key=lambda x: int(x.split('.')[0]))
        ##########必要に応じて、処理するファイルの範囲を指定########
        file_list = file_list[210:211]
        ############################################################

    #for method in ['gradcam', 'gradcampp', 'eigencam']:
    #for method in ['gradcam', 'gradcampp']:
    for method in ['gradcampp']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            folder_main(img_path, method, file_list)
        else:
            main(img_path, method)

In [ ]:
#################################
## 画像全体（bbox外）も表示
#################################

import os
import time
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

import torch
import torch.nn.functional as F
import torchvision
import torch.nn as nn
from utils.general import xywh2xyxy
from utils.datasets import letterbox
from utils.metrics import box_iou
from models.experimental import attempt_load

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")
        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()
        if names is None:
            print('[INFO] fetching names from coco file')
            self.names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                          'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                          'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
                          'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
                          'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',
                          'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
                          'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                          'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
                          'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
        else:
            self.names = names

        # preventing cold start
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            if not x.shape[0]:
                continue

            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            box = xywh2xyxy(x[:, :4])

            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            elif n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence

            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:  # limit detections
                i = i[:max_det]
            if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
                iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
                weights = iou * scores[None]  # box weights
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
                if redundant:
                    i = i[iou.sum(1) > 1]  # require redundancy

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break  # time limit exceeded

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                for *xyxy, conf, cls in det:
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(img_size, xyxy[2])
                    xyxy[3] = min(img_size, xyxy[3])

                    bbox = Box.box2box(xyxy,
                                       in_source=Box.BoxSource.Torch,
                                       to_source=Box.BoxSource.Numpy,
                                       return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img

def apply_heatmap_to_entire_image(mask, image):
    mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
    heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    n_heatmap = (heatmap / 255).astype(np.float32)
    image = image / 255
    res_img = cv2.addWeighted(image, 0.5, n_heatmap, 0.5, 0)  ###ここでヒートマップと元画像のマージの比率を決める
    return res_img

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        res_img = apply_heatmap_to_entire_image(mask, res_img)
    return res_img, None

def put_text_box(bbox, cls_name, res_img): ##マージしているレイヤー名
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')

    # バウンディングボックスを太く描画
    thickness = 3  # 線の太さを増やす（必要に応じて調整してください）
    res_img = Box.put_box(res_img, bbox, thickness=thickness)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.0
    color = (0, 255, 0)  # Green color in BGR format
    text_thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, text_thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, text_thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640), method='gradcam'):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.method = method
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        if self.method == 'eigencam':
            saliency_map = self._eigencam()
            saliency_maps.append(saliency_map)
        else:
            for cls, cls_name in zip(preds[1][0], preds[2][0]):
                if class_idx:
                    score = logits[0][0][cls]
                else:
                    score = logits[0][0].max()
                self.model.zero_grad()
                score.backward(retain_graph=True)
                gradients = self.gradients['value']
                activations = self.activations['value']
                b, k, u, v = gradients.size()

                if self.method == 'gradcam':
                    weights = self._gradcam_weights(gradients, b, k)
                elif self.method == 'gradcampp':
                    weights = self._gradcampp_weights(gradients, activations, score, b, k, u, v)

                saliency_map = (weights * activations).sum(1, keepdim=True)
                saliency_map = F.relu(saliency_map)
                saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
                saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
                saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
                saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def _gradcam_weights(self, gradients, b, k):
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        return weights

    def _gradcampp_weights(self, gradients, activations, score, b, k, u, v):
        alpha_num = gradients.pow(2)
        alpha_denom = gradients.pow(2).mul(2) + \
            activations.mul(gradients.pow(3)).view(b, k, u*v).sum(-1, keepdim=True).view(b, k, 1, 1)
        alpha_denom = torch.where(alpha_denom != 0.0, alpha_denom, torch.ones_like(alpha_denom))
        alpha = alpha_num.div(alpha_denom + 1e-7)

        relu_grad = F.relu(score.exp() * gradients)
        weights = (relu_grad * alpha).view(b, k, u*v).sum(-1).view(b, k, 1, 1)

        return weights

    def _eigencam(self):
        activations = self.activations['value']
        b, k, u, v = activations.size()
        activations_reshaped = activations.view(b, k, -1)

        cov = activations_reshaped @ activations_reshaped.transpose(-1, -2)
        cov /= (u * v)

        eigenvalues, eigenvectors = torch.linalg.eigh(cov)

        leading_eigenvector = eigenvectors[..., -1].unsqueeze(-1)

        eigen_cam = (activations_reshaped.transpose(-1, -2) @ leading_eigenvector).view(b, u, v)
        eigen_cam = F.relu(eigen_cam)

        eigen_cam_min, eigen_cam_max = eigen_cam.min(), eigen_cam.max()
        eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min + 1e-8)

        eigen_cam = F.interpolate(eigen_cam.unsqueeze(1), size=(640, 640), mode='bilinear', align_corners=False)

        return eigen_cam

    def __call__(self, input_img):
        return self.forward(input_img)



def main(img_path, method):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])

    #target_layers = ['model_17_cv3_conv', 'model_20_cv3_conv', 'model_23_cv3_conv']
    target_layers = ['model_24_m_0', 'model_24_m_1', 'model_24_m_2'] ##
    images = [img]  # 元画像を追加

    for layer_name in target_layers:
        saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name, img_size=input_size, method=method)
        tic = time.time()
        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"{layer_name} time:", round(time.time() - tic, 4))
        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr

        res_img = result.copy()
        if len(masks) > 0:  # masksが空でない場合のみ処理を行う
            for j, mask in enumerate(masks[0]):
                bbox = boxes[0][j]
                res_img, _ = get_res_img(bbox, [mask], res_img)
                res_img = put_text_box(bbox, f"{cls_names[0]} - {method} - {layer_name}", res_img)
        images.append(res_img)

    final_image = concat_images(images)
    img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
    output_path = f'{output_dir}/{img_name}'
    os.makedirs(output_dir, exist_ok=True)
    print(f'[INFO] Saving the final image at {output_path}')
    #cv2_imshow(final_image)
    cv2.imwrite(output_path, final_image)


def folder_main(folder_path, method, file_list):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    for index, item in enumerate(file_list, start=1):
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image {index}: {img_basename}")
        img = cv2.imread(img_path)

        # 元の画像の縦横比を維持しながらリサイズ
        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        resized_img = cv2.resize(img, (new_w, new_h))

        # パディングを追加して640×640pxにする
        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        padded_img = cv2.copyMakeBorder(resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(padded_img[..., ::-1])

        #target_layers = ['model_17_cv3_conv', 'model_20_cv3_conv', 'model_23_cv3_conv'] ##
        target_layers = ['model_17_cv3_act', 'model_20_cv3_act', 'model_23_cv3_act'] ##
        ##target_layers = ['model_17_m_0_cv2_conv', 'model_20_m_0_cv2_conv', 'model_23_m_0_cv2_conv']
        #target_layers = ['model_24_m_0', 'model_24_m_1', 'model_24_m_2'] ##
        images = [padded_img]  # 元画像を追加

        for layer_name in target_layers:
            saliency_method = YOLOV5GradCAM(model=model, layer_name=layer_name, img_size=input_size, method=method)
            masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
            print(f"boxes: {boxes}")

            result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
            result = result[..., ::-1]  # convert to bgr

            res_img = result.copy()
            if len(masks) > 0:  # masksが空でない場合のみ処理を行う
                for j, mask in enumerate(masks[0]):
                    bbox = boxes[0][j]
                    res_img, _ = get_res_img(bbox, [mask], res_img)
                    res_img = put_text_box(bbox, f"{cls_names[0]}-{'layer'+layer_name.replace('model_', '')}", res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix=f'-res-{method}')
        output_path = f'{output_dir}/{img_name}'
        os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')
        cv2_imshow(final_image) ##画像を表示
        #cv2.imwrite(output_path, final_image) ##画像を保存


        # メモリ解放
        del saliency_method, masks, logits, boxes, images, final_image
        gc.collect()

if __name__ == '__main__':
    model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
    device = 'cpu'  # 'cuda' または 'cpu'
    img_size = 640  # 入力画像サイズ

    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
    #img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_slit_layer24'  # 出力ディレクトリ
    #output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam++_img_sumaho_layer24'  # 出力ディレクトリ

    # ファイル名を数字でソート
    if os.path.isdir(img_path):
        file_list = sorted(os.listdir(img_path), key=lambda x: int(x.split('.')[0]))
        ##########必要に応じて、処理するファイルの範囲を指定########
        file_list = file_list[210:211]
        ############################################################

    #for method in ['gradcam', 'gradcampp', 'eigencam']:
    #for method in ['gradcam', 'gradcampp']:
    for method in ['gradcampp']:
        print(f"{method.upper()}を実行中:")
        if os.path.isdir(img_path):
            folder_main(img_path, method, file_list)
        else:
            main(img_path, method)

In [ ]:
import gc
gc.collect()

108

In [ ]:
input_size = (640,640)
model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
print(model)

#**Sort GradCam Images**

GradCAM画像をクラスごとにフォルダ分け

In [ ]:
import pandas as pd

# CSVファイルのパス
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara234.csv"

# CSVファイルを読み込み
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

# データフレームの最初の数行を表示
df.head()



In [ ]:
from google.colab import drive
import os
import shutil
import pandas as pd
from tqdm import tqdm

# Mount Google Drive
drive.mount('/gdrive')

# Load the Excel file
file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.xlsx'
df = pd.read_excel(file_path)

# Define source and destination directories
source_dirs = [
    '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit',
    '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_sumaho'
]
destination_base_dirs = [
    '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_slit_sorted',
    '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_sumaho_sorted'
]

# Iterate over the source and destination directory pairs
for source_dir, destination_base_dir in zip(source_dirs, destination_base_dirs):
    print(f'Processing files from {source_dir} to {destination_base_dir}')
    # Iterate over the dataframe and copy files to the appropriate class folder
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        serial_number = row['serial_number']
        class_num = row['class_num']
        source_file = os.path.join(source_dir, f'{serial_number}-res.jpg')
        destination_dir = os.path.join(destination_base_dir, str(class_num))

        # Create the destination directory if it does not exist
        os.makedirs(destination_dir, exist_ok=True)

        # Copy the file to the destination directory
        destination_file = os.path.join(destination_dir, f'{serial_number}-res.jpg')
        if os.path.exists(source_file):
            shutil.copy(source_file, destination_file)
        else:
            print(f'File {source_file} does not exist.')

print('Files have been copied successfully.')


Mounted at /gdrive
Processing files from /gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit to /gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_slit_sorted


100%|██████████| 228/228 [00:02<00:00, 88.10it/s]


Processing files from /gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_sumaho to /gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_sumaho_sorted


100%|██████████| 228/228 [00:02<00:00, 89.16it/s]

Files have been copied successfully.


#**Compare area of interest**

注目度n%以上の画像を

###**ニ値化して表示**

In [ ]:
#################
#################

import numpy as np
from deep_utils.utils.box_utils.boxes import Box
import torch
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
import cv2
import time
import torchvision
import torch.nn as nn
from utils.metrics import box_iou

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")
        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()
        # fetch the names
        if names is None:
            print('[INFO] fetching names from coco file')
            self.names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                          'traffic light',
                          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                          'cow',
                          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase',
                          'frisbee',
                          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                          'surfboard',
                          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
                          'apple',
                          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                          'couch',
                          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                          'keyboard', 'cell phone',
                          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                          'teddy bear',
                          'hair drier', 'toothbrush']
        else:
            self.names = names

        # preventing cold start
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        """Runs Non-Maximum Suppression (NMS) on inference and logits results

        Returns:
             list of detections, on (n,6) tensor per image [xyxy, conf, cls] and pruned input logits (n, number-classes)
        """

        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            # Apply constraints
            # x[((x[..., 2:4] < min_wh) | (x[..., 2:4] > max_wh)).any(1), 4] = 0  # width-height
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            # Cat apriori labels if autolabelling
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            # If none remain process next image
            if not x.shape[0]:
                continue

            # Compute conf
            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            # log_ *= x[:, 4:5]
            # Box (center x, center y, width, height) to (x1, y1, x2, y2)
            box = xywh2xyxy(x[:, :4])

            # Detections matrix nx6 (xyxy, conf, cls)
            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                # log_ = x[:, 5:]
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            # Filter by class
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            # Check shape
            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            elif n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence

            # Batched NMS
            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:  # limit detections
                i = i[:max_det]
            if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
                # update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
                iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
                weights = iou * scores[None]  # box weights
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
                if redundant:
                    i = i[iou.sum(1) > 1]  # require redundancy

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break  # time limit exceeded

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):

        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                for *xyxy, conf, cls in det:
                    # xyxyの値を修正
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(img_size, xyxy[2])
                    xyxy[3] = min(img_size, xyxy[3])

                    bbox = Box.box2box(xyxy,
                                       in_source=Box.BoxSource.Torch,
                                       to_source=Box.BoxSource.Numpy,
                                       return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img


In [ ]:
import os
import time
import numpy as np
# from models.gradcam import YOLOV5GradCAM
#from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc



# パラメータ
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
#img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial/99.jpg"
#output_dir = 'out'  # 出力ディレクトリ
output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'  # 出力ディレクトリ
img_size = 640  # 入力画像サイズ
target_layer = 'model_23_cv3_act'  # GradCAM を適用するレイヤー
#target_layer = 'model_23_m_0_cv2_act'  # GradCAM を適用するレイヤー
#target_layer = 'model_24_m_2'  # GradCAM を適用するレイヤー
method = 'gradcam'  # 'gradcam' または 'gradcampp'
device = 'cpu'  # 'cuda' または 'cpu'

# クラス名リストを指定
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]


import time
import torch
import torch.nn.functional as F

def get_res_img(bbox, masks, res_img, threshold):
    total_intersect_pixels = 0  # 共通部分のピクセル数のカウンタを初期化

    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        mask[mask < threshold*255] = 0  # 追加: 128未満の値を0にする
        mask[mask >= threshold*255] = 255  # 追加: 128未満の値を0にする
        binary_mask = mask >= threshold * 255  # 閾値を超える部分を二値化
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())

        # bboxの範囲内のマスクの部分を取得
        x1, y1, x2, y2 = bbox
        mask_bbox = binary_mask[y1:y2, x1:x2]

        # 閾値を超える共通部分のピクセル数をカウント
        intersect_pixels = np.sum(mask_bbox)
        total_intersect_pixels += intersect_pixels

        # mask_bbox のピクセル数を取得
        mask_bbox_area = mask_bbox.size

        # AOI を計算
        AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0
        print(f"Area of Interest (AOI): {AOI}")

    return res_img, n_heatmat


def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    # this is a bug in cv2. It does not put box on a converted image from torch unless it's buffered and read again!
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    # Set the font size and other parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)  # Green color in BGR format
    thickness = 2

    # Calculate the text size and position
    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10  # Adjust the horizontal position to place the text near the left edge
    text_y = text_size[1] + 10  # Adjust the vertical position to place the text near the top edge

    # Put the text on the image
    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def concat_images(images):
    w, h = images[0].shape[:2]
    width = w
    height = h * len(images)
    base_img = np.zeros((width, height, 3), dtype=np.uint8)
    for i, img in enumerate(images):
        base_img[:, h * i:h * (i + 1), ...] = img
    return base_img

def find_yolo_layer(model, layer_name):
    """Find yolov5 layer to calculate GradCAM and GradCAM++

    Args:
        model: yolov5 model.
        layer_name (str): the name of layer with its hierarchical information.

    Return:
        target_layer: found layer
    """
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer


class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640)):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.cls_names = []  # 追加: Top1~3のクラス名を保存するリスト

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))
        # print('[INFO] saliency_map size :', self.activations['value'].shape[2:])

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        tic = time.time()
        preds, logits = self.model(input_img)
        # print("[INFO] model-forward took: ", round(time.time() - tic, 4), 'seconds')

        _, top3_indices = torch.topk(logits[0], k=3)

        if top3_indices.numel() > 0:
            preds[1][0] = top3_indices.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]  # 修正: Top1~3のクラス名を保存
        else:
            self.cls_names = []  # top3_indicesが空の場合は空のリストを代入

        for cls, cls_name in zip(preds[1][0], preds[2][0]):
            if class_idx:
                score = logits[0][0][cls]
            else:
                score = logits[0][0].max()
            self.model.zero_grad()
            tic = time.time()
            score.backward(retain_graph=True)
            # print(f"[INFO] {cls_name}, model-backward took: ", round(time.time() - tic, 4), 'seconds')
            gradients = self.gradients['value']
            activations = self.activations['value']
            b, k, u, v = gradients.size()
            alpha = gradients.view(b, k, -1).mean(2)
            weights = alpha.view(b, k, 1, 1)
            saliency_map = (weights * activations).sum(1, keepdim=True)
            saliency_map = F.relu(saliency_map)
            saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
            saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
            saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
            saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names


    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size,
                                      names=names)
    torch_img = model.preprocessing(img[..., ::-1])
    if method == 'gradcam':
        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)
    tic = time.time()
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    print("total time:", round(time.time() - tic, 4))
    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]  # convert to bgr
    images = [result]

    if len(masks) > 0:
        mask = masks[0][0]  # top1のマスクのみ使用
        bbox = boxes[0][0]  # top1のバウンディングボックスのみ使用
        cls_name = cls_names[0]  # top1のクラス名のみ使用
        res_img = result.copy()
        res_img, _ = get_res_img(bbox, [mask], res_img, GradCAM_threshold)
        res_img = put_text_box(bbox, cls_name, res_img)
        images.append(res_img)

    final_image = concat_images(images)
    img_name = split_extension(os.path.split(img_path)[-1], suffix='-res')
    output_path = f'{output_dir}/{img_name}'
    #os.makedirs(output_dir, exist_ok=True)
    print(f'[INFO] Saving the final image at {output_path}')
    cv2_imshow(final_image)  # cv2.imwriteの代わりにcv2_imshowを使用



def folder_main(folder_path):
    input_size = (img_size, img_size)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

    ##############################################################################################
    ######## ファイル名を数字でソート#############################################################
    file_list = sorted(os.listdir(folder_path), key=lambda x: int(x.split('.')[0]))[0:5]
    ##############################################################################################
    ##############################################################################################

    for item in file_list:
        img_path = os.path.join(folder_path, item)
        img_basename = os.path.basename(img_path)
        print(f"Processing image: {img_basename}")
        img = cv2.imread(img_path)

        # 元の画像の縦横比を維持しながらリサイズ
        h, w = img.shape[:2]
        if h > w:
            new_h, new_w = img_size, int(img_size * w / h)
        else:
            new_h, new_w = int(img_size * h / w), img_size
        img = cv2.resize(img, (new_w, new_h))

        # パディングを追加して640×640pxにする
        delta_w = img_size - new_w
        delta_h = img_size - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

        torch_img = model.preprocessing(img[..., ::-1])

        if method == 'gradcam':
            saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)

        masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
        print(f"boxes: {boxes}")

        result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
        result = result[..., ::-1]  # convert to bgr
        images = [result]

        if len(masks) > 0:
            mask = masks[0][0]  # top1のマスクのみ使用
            bbox = boxes[0][0]  # top1のバウンディングボックスのみ使用
            cls_name = cls_names[0]  # top1のクラス名のみ使用
            res_img = result.copy()
            res_img, _ = get_res_img(bbox, [mask], res_img, GradCAM_threshold)
            res_img = put_text_box(bbox, cls_name, res_img)
            images.append(res_img)

        final_image = concat_images(images)
        img_name = split_extension(os.path.split(img_path)[-1], suffix='-res')
        output_path = f'{output_dir}/{img_name}'
        #os.makedirs(output_dir, exist_ok=True)
        print(f'[INFO] Saving the final image at {output_path}')
        cv2_imshow(final_image)  # cv2.imwriteの代わりにcv2_imshowを使用

        # オブジェクトを明示的に削除
        del saliency_method
        del masks
        del logits
        del boxes
        del images
        del final_image
        gc.collect()


if __name__ == '__main__':
    #target_layer = 'model_17_cv3_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_20_cv3_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_23_cv3_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_23_cv3_act'  # GradCAM を適用するレイヤー  →デフォルト
    #target_layer = 'model_23_m_0_cv2_conv'  # GradCAM を適用するレイヤー
    #target_layer = 'model_23_m_0_cv2_act'  # GradCAM を適用するレイヤー
    #target_layer = 'model_24_m_0'  # GradCAM を適用するレイヤー
    #target_layer = 'model_24_m_1'  # GradCAM を適用するレイヤー
    target_layer = 'model_24_m_2'  # GradCAM を適用するレイヤー
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
    #img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'  # 出力ディレクトリ
    #output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'  # 出力ディレクトリ

    GradCAM_threshold = 0.5 #GradCAMの閾値設定
    if os.path.isdir(img_path):
        folder_main(img_path)
    else:
        main(img_path)

###**Area of interestの計算**

結果をcsvに保存する

In [13]:
import pandas as pd

# CSVファイルのパス
input_file_path = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルをDataFrameとして読み込む
df = pd.read_csv(input_file_path)

# 'Unnamed'が含まれる列を削除する
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 例としてthresholdの値を設定
threshold = 0.5

# 指定されたレイヤー
layers = ["24_m_0", "24_m_1", "24_m_2"]

# 各レイヤーに対して新しい列を作成
for layer in layers:
    df[f'AOI_{threshold}_layer{layer}'] = None  # 初期値を設定

# 修正後のDataFrameを表示する
print(df.head())

# DataFrameをCSVファイルとして保存
df.to_csv(file_path, index=False)

print(f"Updated DataFrame has been saved to: {file_path}")


   serial_number    basename disease_English  class_num  top1  ...  \
0              1   9TTR_R_01       infection          0     0  ...   
1              2  12TTR_R_01       infection          0     0  ...   
2              3  13TTR_R_01       infection          0     2  ...   
3              4  21TTR_L_01       infection          0     0  ...   
4              5  25TTR_R_01            APAC          6     6  ...   

   AOI_0.5_layer20  AOI_0.5_layer23  AOI_0.5_layer24_m_0  AOI_0.5_layer24_m_1  \
0         0.000000         0.127202                 None                 None   
1         0.000000         0.179596                 None                 None   
2         0.012137         0.204253                 None                 None   
3         0.411027         0.027142                 None                 None   
4         0.000000         0.119527                 None                 None   

   AOI_0.5_layer24_m_2  
0                 None  
1                 None  
2                

,serial_number,basename,disease_English,class_num,top1,top1_prob,top2,top2_prob,top3,top3_prob,AOI_0.5,AOI_0.5_layer17,AOI_0.5_layer20,AOI_0.5_layer23,AOI_0.5_24_m_0,AOI_0.5_24_m_1,AOI_0.5_24_m_2
0,1,9TTR_R_01,infection,0,0,0.936966,4,0.010804,0,0.006100,0.217295,0.000000,0.000000,0.127202,None,None,None
1,2,12TTR_R_01,infection,0,0,0.922136,2,0.001290,3,0.000831,0.179596,0.160531,0.000000,0.179596,None,None,None
2,3,13TTR_R_01,infection,0,2,0.426648,8,0.154254,0,0.099206,0.204253,0.089683,0.012137,0.204253,None,None,None
3,4,21TTR_L_01,infection,0,0,0.689450,2,0.456746,3,0.015478,0.027142,0.000000,0.411027,0.027142,None,None,None
4,5,25TTR_R_01,APAC,6,6,0.912041,0,0.065191,4,0.060999,0.119527,0.143779,0.000000,0.119527,None,None,None


In [14]:
#################
#################

import numpy as np
from deep_utils.utils.box_utils.boxes import Box
import torch
from models.experimental import attempt_load
from utils.general import xywh2xyxy
from utils.datasets import letterbox
import cv2
import time
import torchvision
import torch.nn as nn
from utils.metrics import box_iou

class YOLOV5TorchObjectDetector(nn.Module):
    def __init__(self,
                 model_weight,
                 device,
                 img_size,
                 names=None,
                 mode='eval',
                 confidence=0.25,
                 iou_thresh=0.45,
                 agnostic_nms=False):
        super(YOLOV5TorchObjectDetector, self).__init__()
        self.device = device
        self.model = None
        self.img_size = img_size
        self.mode = mode
        self.confidence = confidence
        self.iou_thresh = iou_thresh
        self.agnostic = agnostic_nms
        self.model = attempt_load(model_weight, device=device)
        print("[INFO] Model is loaded")
        self.model.requires_grad_(True)
        self.model.to(device)
        if self.mode == 'train':
            self.model.train()
        else:
            self.model.eval()
        # fetch the names
        if names is None:
            print('[INFO] fetching names from coco file')
            self.names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                          'traffic light',
                          'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
                          'cow',
                          'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase',
                          'frisbee',
                          'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
                          'surfboard',
                          'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
                          'apple',
                          'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                          'couch',
                          'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                          'keyboard', 'cell phone',
                          'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                          'teddy bear',
                          'hair drier', 'toothbrush']
        else:
            self.names = names

        # preventing cold start
        img = torch.zeros((1, 3, *self.img_size), device=device)
        self.model(img)

    @staticmethod
    def non_max_suppression(prediction, logits, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False,
                            multi_label=False, labels=(), max_det=300):
        """Runs Non-Maximum Suppression (NMS) on inference and logits results

        Returns:
             list of detections, on (n,6) tensor per image [xyxy, conf, cls] and pruned input logits (n, number-classes)
        """

        nc = prediction.shape[2] - 5  # number of classes
        xc = prediction[..., 4] > conf_thres  # candidates

        # Checks
        assert 0 <= conf_thres <= 1, f'Invalid Confidence threshold {conf_thres}, valid values are between 0.0 and 1.0'
        assert 0 <= iou_thres <= 1, f'Invalid IoU {iou_thres}, valid values are between 0.0 and 1.0'

        # Settings
        min_wh, max_wh = 2, 4096  # (pixels) minimum and maximum box width and height
        max_nms = 30000  # maximum number of boxes into torchvision.ops.nms()
        time_limit = 10.0  # seconds to quit after
        redundant = True  # require redundant detections
        multi_label &= nc > 1  # multiple labels per box (adds 0.5ms/img)
        merge = False  # use merge-NMS

        t = time.time()
        output = [torch.zeros((0, 6), device=prediction.device)] * prediction.shape[0]
        logits_output = [torch.zeros((0, 80), device=logits.device)] * logits.shape[0]
        for xi, (x, log_) in enumerate(zip(prediction, logits)):  # image index, image inference
            # Apply constraints
            # x[((x[..., 2:4] < min_wh) | (x[..., 2:4] > max_wh)).any(1), 4] = 0  # width-height
            x = x[xc[xi]]  # confidence
            log_ = log_[xc[xi]]
            # Cat apriori labels if autolabelling
            if labels and len(labels[xi]):
                l = labels[xi]
                v = torch.zeros((len(l), nc + 5), device=x.device)
                v[:, :4] = l[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(l)), l[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)

            # If none remain process next image
            if not x.shape[0]:
                continue

            # Compute conf
            x[:, 5:] *= x[:, 4:5]  # conf = obj_conf * cls_conf
            # log_ *= x[:, 4:5]
            # Box (center x, center y, width, height) to (x1, y1, x2, y2)
            box = xywh2xyxy(x[:, :4])

            # Detections matrix nx6 (xyxy, conf, cls)
            if multi_label:
                i, j = (x[:, 5:] > conf_thres).nonzero(as_tuple=False).T
                x = torch.cat((box[i], x[i, j + 5, None], j[:, None].float()), 1)
            else:  # best class only
                conf, j = x[:, 5:].max(1, keepdim=True)
                # log_ = x[:, 5:]
                x = torch.cat((box, conf, j.float()), 1)[conf.view(-1) > conf_thres]
                log_ = log_[conf.view(-1) > conf_thres]
            # Filter by class
            if classes is not None:
                x = x[(x[:, 5:6] == torch.tensor(classes, device=x.device)).any(1)]

            # Check shape
            n = x.shape[0]  # number of boxes
            if not n:  # no boxes
                continue
            elif n > max_nms:  # excess boxes
                x = x[x[:, 4].argsort(descending=True)[:max_nms]]  # sort by confidence

            # Batched NMS
            c = x[:, 5:6] * (0 if agnostic else max_wh)  # classes
            boxes, scores = x[:, :4] + c, x[:, 4]  # boxes (offset by class), scores
            i = torchvision.ops.nms(boxes, scores, iou_thres)  # NMS
            if i.shape[0] > max_det:  # limit detections
                i = i[:max_det]
            if merge and (1 < n < 3E3):  # Merge NMS (boxes merged using weighted mean)
                # update boxes as boxes(i,4) = weights(i,n) * boxes(n,4)
                iou = box_iou(boxes[i], boxes) > iou_thres  # iou matrix
                weights = iou * scores[None]  # box weights
                x[i, :4] = torch.mm(weights, x[:, :4]).float() / weights.sum(1, keepdim=True)  # merged boxes
                if redundant:
                    i = i[iou.sum(1) > 1]  # require redundancy

            output[xi] = x[i]
            logits_output[xi] = log_[i]
            assert log_[i].shape[0] == x[i].shape[0]
            if (time.time() - t) > time_limit:
                print(f'WARNING: NMS time limit {time_limit}s exceeded')
                break  # time limit exceeded

        return output, logits_output

    @staticmethod
    def yolo_resize(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):

        return letterbox(img, new_shape=new_shape, color=color, auto=auto, scaleFill=scaleFill, scaleup=scaleup)

    def forward(self, img):
        prediction, logits, _ = self.model(img, augment=False)
        prediction, logits = self.non_max_suppression(prediction, logits, self.confidence, self.iou_thresh,
                                                      classes=None,
                                                      agnostic=self.agnostic)
        self.boxes, self.class_names, self.classes, self.confidences = [[[] for _ in range(img.shape[0])] for _ in
                                                                        range(4)]
        for i, det in enumerate(prediction):  # detections per image
            if len(det):
                for *xyxy, conf, cls in det:
                    # xyxyの値を修正
                    xyxy[0] = max(0, xyxy[0])
                    xyxy[1] = max(0, xyxy[1])
                    xyxy[2] = min(self.img_size[0], xyxy[2])
                    xyxy[3] = min(self.img_size[1], xyxy[3])

                    bbox = Box.box2box(xyxy,
                                      in_source=Box.BoxSource.Torch,
                                      to_source=Box.BoxSource.Numpy,
                                      return_int=True)
                    self.boxes[i].append(bbox)
                    self.confidences[i].append(round(conf.item(), 2))
                    cls = int(cls.item())
                    self.classes[i].append(cls)
                    if self.names is not None:
                        self.class_names[i].append(self.names[cls])
                    else:
                        self.class_names[i].append(cls)
        return [self.boxes, self.classes, self.class_names, self.confidences], logits

    def preprocessing(self, img):
        if len(img.shape) != 4:
            img = np.expand_dims(img, axis=0)
        im0 = img.astype(np.uint8)
        img = np.array([self.yolo_resize(im, new_shape=self.img_size)[0] for im in im0])
        img = img.transpose((0, 3, 1, 2))
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img / 255.0
        return img


In [17]:
import gc
gc.collect()

33

In [ ]:
# #################
# # 最終（23）layerのみの解析
# #################

# import pandas as pd
# import numpy as np
# import cv2
# from google.colab.patches import cv2_imshow
# from deep_utils import Box, split_extension
# import os
# from tqdm import tqdm
# #from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
# # from models.gradcam import YOLOV5GradCAM
# import time
# import torch
# import torch.nn.functional as F
# import gc

# class YOLOV5GradCAM:
#     def __init__(self, model, layer_name, img_size=(640, 640)):
#         self.model = model
#         self.gradients = dict()
#         self.activations = dict()
#         self.cls_names = []  # 追加: Top1~3のクラス名を保存するリスト

#         def backward_hook(module, grad_input, grad_output):
#             self.gradients['value'] = grad_output[0]
#             return None

#         def forward_hook(module, input, output):
#             self.activations['value'] = output
#             return None

#         target_layer = find_yolo_layer(self.model, layer_name)
#         target_layer.register_forward_hook(forward_hook)
#         target_layer.register_backward_hook(backward_hook)

#         device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
#         self.model(torch.zeros(1, 3, *img_size, device=device))
#         # print('[INFO] saliency_map size :', self.activations['value'].shape[2:])

#     def forward(self, input_img, class_idx=True):
#         saliency_maps = []
#         b, c, h, w = input_img.size()
#         tic = time.time()
#         preds, logits = self.model(input_img)
#         # print("[INFO] model-forward took: ", round(time.time() - tic, 4), 'seconds')

#         _, top3_indices = torch.topk(logits[0], k=3)

#         if top3_indices.numel() > 0:
#             preds[1][0] = top3_indices.tolist()[0]
#             preds[2][0] = [names[i] for i in preds[1][0]]
#             self.cls_names = preds[2][0]  # 修正: Top1~3のクラス名を保存
#         else:
#             self.cls_names = []  # top3_indicesが空の場合は空のリストを代入

#         for cls, cls_name in zip(preds[1][0], preds[2][0]):
#             if class_idx:
#                 score = logits[0][0][cls]
#             else:
#                 score = logits[0][0].max()
#             self.model.zero_grad()
#             tic = time.time()
#             score.backward(retain_graph=True)
#             # print(f"[INFO] {cls_name}, model-backward took: ", round(time.time() - tic, 4), 'seconds')
#             gradients = self.gradients['value']
#             activations = self.activations['value']
#             b, k, u, v = gradients.size()
#             alpha = gradients.view(b, k, -1).mean(2)
#             weights = alpha.view(b, k, 1, 1)
#             saliency_map = (weights * activations).sum(1, keepdim=True)
#             saliency_map = F.relu(saliency_map)
#             saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
#             saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
#             saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
#             saliency_maps.append(saliency_map)

#         return saliency_maps, logits, preds, self.cls_names


#     def __call__(self, input_img):
#         return self.forward(input_img)

# def find_yolo_layer(model, layer_name):
#     """Find yolov5 layer to calculate GradCAM and GradCAM++

#     Args:
#         model: yolov5 model.
#         layer_name (str): the name of layer with its hierarchical information.

#     Return:
#         target_layer: found layer
#     """
#     hierarchy = layer_name.split('_')
#     target_layer = model.model._modules[hierarchy[0]]

#     for h in hierarchy[1:]:
#         target_layer = target_layer._modules[h]
#     return target_layer

# def get_aoi(bbox, masks, threshold):
#     total_intersect_pixels = 0  # 共通部分のピクセル数のカウンタを初期化

#     for mask in masks:
#         # マスクの前処理
#         mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
#         mask[mask < threshold * 255] = 0
#         mask[mask >= threshold * 255] = 255
#         binary_mask = mask >= threshold * 255  # 閾値を超える部分を二値化

#         # bboxの範囲内のマスク部分を取得
#         x1, y1, x2, y2 = bbox
#         mask_bbox = binary_mask[y1:y2, x1:x2]

#         # 閾値を超える共通部分のピクセル数をカウント
#         intersect_pixels = np.sum(mask_bbox)
#         total_intersect_pixels += intersect_pixels

#         # mask_bbox のピクセル数を取得
#         mask_bbox_area = (y2 - y1) * (x2 - x1)

#         # AOI を計算
#         AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

#     # if len(bbox) == 0 or len(masks) == 0:
#     #     return 0.0

#     # for mask in masks:
#     #     mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
#     #     mask[mask < threshold*255] = 0
#     #     mask[mask >= threshold*255] = 255
#     #     heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
#     #     n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
#     #     aoi = np.sum(n_heatmat > 0) / (n_heatmat.shape[0] * n_heatmat.shape[1])
#     #     print(f"Area of Interest (original_AOI): {aoi}")

#     return AOI

# def calculate_aoi(folder_path, csv_path, threshold, model, saliency_method, start_index=0, end_index=None):
#     df = pd.read_csv(csv_path)
#     df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

#     if f'AOI_{threshold}' not in df.columns:
#         df[f'AOI_{threshold}'] = None

#     if end_index is None:
#         end_index = len(df)

#     for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
#         if index < start_index or index >= end_index:
#             continue

#         serial_number = row['serial_number']

#         if pd.isna(serial_number):
#             print(f"Skipping row {index} due to NaN serial_number")
#             continue

#         img_name = f"{int(serial_number)}.jpg"
#         img_path = os.path.join(folder_path, img_name)

#         # Rest of the code remains the same
#         if os.path.exists(img_path):
#             img = cv2.imread(img_path)
#             torch_img = model.preprocessing(img[..., ::-1])

#             try:
#                 masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
#             except ValueError as e:
#                 print(f"Error processing image {img_name}: {str(e)}")
#                 continue

#             if len(masks) > 0 and len(boxes) > 0:
#                 mask = masks[0][0]  # top1のマスクのみ使用
#                 bbox = boxes[0][0]  # top1のバウンディングボックスのみ使用

#                 aoi = get_aoi(bbox, [mask], threshold)
#                 df.at[index, f'AOI_{threshold}'] = aoi

#                 print(f"Image: {img_name}, AOI: {aoi}")
#         else:
#             print(f"Image not found: {img_name}")

#     df.to_csv(csv_path, index=False)

# # 使用例
# folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
# csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv"
# #folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
# #csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv"
# threshold = 0.5
# start_index = 51 # 開始するインデックスを指定
# end_index = 100 # 終了するインデックスを指定

# # モデルとsaliency_methodの定義
# model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
# device = 'cpu'
# input_size = (640, 640)
# names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

# model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
# target_layer = 'model_23_cv3_conv'
# saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)

# calculate_aoi(folder_path, csv_path, threshold, model, saliency_method, start_index, end_index)

In [ ]:
import gc
gc.collect()

4

In [16]:
##################
##################
## 3-layerで解析##
##################
##################

import pandas as pd
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import os
from tqdm import tqdm
#from models.yolo_v5_object_detector import YOLOV5TorchObjectDetector
# from models.gradcam import YOLOV5GradCAM
import time
import torch
import torch.nn.functional as F
import gc

class YOLOV5GradCAM:
   def __init__(self, model, layer_name, img_size=(640, 640)):
       self.model = model
       self.gradients = dict()
       self.activations = dict()
       self.cls_names = []  # 追加: Top1~3のクラス名を保存するリスト

       def backward_hook(module, grad_input, grad_output):
           self.gradients['value'] = grad_output[0]
           return None

       def forward_hook(module, input, output):
           self.activations['value'] = output
           return None

       target_layer = find_yolo_layer(self.model, layer_name)
       target_layer.register_forward_hook(forward_hook)
       target_layer.register_backward_hook(backward_hook)

       device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
       self.model(torch.zeros(1, 3, *img_size, device=device))
       # print('[INFO] saliency_map size :', self.activations['value'].shape[2:])

   def forward(self, input_img, class_idx=True):
       saliency_maps = []
       b, c, h, w = input_img.size()
       tic = time.time()
       preds, logits = self.model(input_img)
       # print("[INFO] model-forward took: ", round(time.time() - tic, 4), 'seconds')

       _, top3_indices = torch.topk(logits[0], k=3)

       if top3_indices.numel() > 0:
           preds[1][0] = top3_indices.tolist()[0]
           preds[2][0] = [names[i] for i in preds[1][0]]
           self.cls_names = preds[2][0]  # 修正: Top1~3のクラス名を保存
       else:
           self.cls_names = []  # top3_indicesが空の場合は空のリストを代入

       for cls, cls_name in zip(preds[1][0], preds[2][0]):
           if class_idx:
               score = logits[0][0][cls]
           else:
               score = logits[0][0].max()
           self.model.zero_grad()
           tic = time.time()
           score.backward(retain_graph=True)
           # print(f"[INFO] {cls_name}, model-backward took: ", round(time.time() - tic, 4), 'seconds')
           gradients = self.gradients['value']
           activations = self.activations['value']
           b, k, u, v = gradients.size()
           alpha = gradients.view(b, k, -1).mean(2)
           weights = alpha.view(b, k, 1, 1)
           saliency_map = (weights * activations).sum(1, keepdim=True)
           saliency_map = F.relu(saliency_map)
           saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
           saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
           saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
           saliency_maps.append(saliency_map)

       return saliency_maps, logits, preds, self.cls_names


   def __call__(self, input_img):
       return self.forward(input_img)

def find_yolo_layer(model, layer_name):
   """Find yolov5 layer to calculate GradCAM and GradCAM++

   Args:
       model: yolov5 model.
       layer_name (str): the name of layer with its hierarchical information.

   Return:
       target_layer: found layer
   """
   hierarchy = layer_name.split('_')
   target_layer = model.model._modules[hierarchy[0]]

   for h in hierarchy[1:]:
       target_layer = target_layer._modules[h]
   return target_layer

def get_aoi(bbox, masks, threshold):
   total_intersect_pixels = 0  # 共通部分のピクセル数のカウンタを初期化

   for mask in masks:
       # マスクの前処理
       mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
       mask[mask < threshold * 255] = 0
       mask[mask >= threshold * 255] = 255
       binary_mask = mask >= threshold * 255  # 閾値を超える部分を二値化

       # bboxの範囲内のマスク部分を取得
       x1, y1, x2, y2 = bbox
       mask_bbox = binary_mask[y1:y2, x1:x2]

       # 閾値を超える共通部分のピクセル数をカウント
       intersect_pixels = np.sum(mask_bbox)
       total_intersect_pixels += intersect_pixels

       # mask_bbox のピクセル数を取得
       mask_bbox_area = (y2 - y1) * (x2 - x1)

       # AOI を計算
       AOI = total_intersect_pixels / mask_bbox_area if mask_bbox_area > 0 else 0

   # if len(bbox) == 0 or len(masks) == 0:
   #     return 0.0

   # for mask in masks:
   #     mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
   #     mask[mask < threshold*255] = 0
   #     mask[mask >= threshold*255] = 255
   #     heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
   #     n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
   #     aoi = np.sum(n_heatmat > 0) / (n_heatmat.shape[0] * n_heatmat.shape[1])
   #     print(f"Area of Interest (original_AOI): {aoi}")

   return AOI

def calculate_aoi(folder_path, csv_path, threshold, model, saliency_methods, start_index=0, end_index=None):
   df = pd.read_csv(csv_path)
   df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

  #  for layer_num in [17, 20, 23]:
  #      if f'AOI_{threshold}_layer{layer_num}' not in df.columns:
  #          df[f'AOI_{threshold}_layer{layer_num}'] = None
   for layer_num in [0, 1, 2]:
       if f'AOI_{threshold}_layer24_m_{layer_num}' not in df.columns:
           df[f'AOI_{threshold}_layer24_m_{layer_num}'] = None

   if end_index is None:
       end_index = len(df)

   for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
       if index < start_index or index >= end_index:
           continue

       serial_number = row['serial_number']

       if pd.isna(serial_number):
           print(f"Skipping row {index} due to NaN serial_number")
           continue

       img_name = f"{int(serial_number)}.jpg"
       img_path = os.path.join(folder_path, img_name)

       # Rest of the code remains the same
       if os.path.exists(img_path):
           img = cv2.imread(img_path)
           torch_img = model.preprocessing(img[..., ::-1])

           for layer_num, saliency_method in zip([0, 1, 2], saliency_methods):
               try:
                   masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
               except ValueError as e:
                   print(f"Error processing image {img_name}: {str(e)}")
                   continue

               if len(masks) > 0 and len(boxes) > 0:
                   mask = masks[0][0]  # top1のマスクのみ使用
                   bbox = boxes[0][0]  # top1のバウンディングボックスのみ使用

                   aoi = get_aoi(bbox, [mask], threshold)
                   #df.at[index, f'AOI_{threshold}_layer{layer_num}'] = aoi
                   df.at[index, f'AOI_{threshold}_layer24_m_{layer_num}'] = aoi

                   print(f"Image: {img_name}, Layer: {layer_num}, AOI: {aoi}")
       else:
           print(f"Image not found: {img_name}")

   df.to_csv(csv_path, index=False)

# 使用例
folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial"
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv"
#folder_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial"
#csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv"
threshold = 0.5
start_index = 0 # 開始するインデックスを指定
end_index = 20 # 終了するインデックスを指定

# モデルとsaliency_methodの定義
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
device = 'cpu'
input_size = (640, 640)
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)

#target_layers = ['model_17_cv3_conv', 'model_20_cv3_conv', 'model_23_cv3_conv']
target_layers = ['model_24_m_0', 'model_24_m_1', 'model_24_m_2']

saliency_methods = [YOLOV5GradCAM(model=model, layer_name=layer, img_size=input_size) for layer in target_layers]

calculate_aoi(folder_path, csv_path, threshold, model, saliency_methods, start_index, end_index)

/content/yolov5-gradcam/models/experimental.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_download(w), map_location=device)
INFO:models.yolo

[INFO] Model is loaded


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be r

Image: 1.jpg, Layer: 0, AOI: 0.0


<ipython-input-16-d127aba138be>:109: RuntimeWarning: invalid value encountered in cast
  mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


...

Image: 20.jpg, Layer: 0, AOI: 0.0


<ipython-input-16-d127aba138be>:109: RuntimeWarning: invalid value encountered in cast
  mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1640: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Image: 20.jpg, Layer: 1, AOI: 0.0


Processing images: 100%|██████████| 228/228 [01:00<00:00,  3.75it/s]

Image: 20.jpg, Layer: 2, AOI: 0.010448783830078794


In [ ]:
# show_result
import pandas as pd

# CSVファイルのパス
csv_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv"

# DataFrameとしてCSVファイルを読み込む
df = pd.read_csv(csv_path)

# DataFrameの最初の数行を表示する
print(df.head())


#**注目点色塗り**

In [ ]:
#注目点に色を塗る（反転あり）
import os
import time
import numpy as np
import cv2
import torch
import torch.nn.functional as F
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

# パラメータ
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial/99.jpg"
output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
img_size = 640
target_layer = 'model_23_cv3_act'
method = 'gradcam'
device = 'cpu'
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640)):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)
        _, top1_idx = torch.topk(logits[0], k=1)

        if top1_idx.numel() > 0:
            preds[1][0] = top1_idx.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        cls, cls_name = preds[1][0][0], preds[2][0][0]
        if class_idx:
            score = logits[0][0][cls]
        else:
            score = logits[0][0].max()
        self.model.zero_grad()
        score.backward(retain_graph=True)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        saliency_map = (weights * activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
        saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, threshold=0.5):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])
    if method == 'gradcam':
        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]

    if masks:
        mask = masks[0].squeeze().cpu().numpy()
        if mask.max() >= threshold:
            binary_mask = np.where(mask >= threshold, 1, 0).astype(np.uint8)
            black_mask = np.where(mask >= threshold, 0, 1).astype(np.uint8)
            binary_mask = cv2.resize(binary_mask, (result.shape[1], result.shape[0]))
            black_mask = cv2.resize(black_mask, (result.shape[1], result.shape[0]))

            color = np.array([0,0,0], dtype=np.uint8) #マスクの色：白は[0,0,0]
            masked_res_img = result * binary_mask[..., np.newaxis] + (1 - binary_mask[..., np.newaxis]) * color
            masked_res_img = (masked_res_img / masked_res_img.max()) * 255

            black_masked_res_img = result * black_mask[..., np.newaxis] + (1 - black_mask[..., np.newaxis]) * color
            black_masked_res_img = (black_masked_res_img / black_masked_res_img.max()) * 255

            cv2_imshow(masked_res_img.astype(np.uint8))
            cv2_imshow(black_masked_res_img.astype(np.uint8))

if __name__ == '__main__':
    target_layer = 'model_23_cv3_conv'
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/8.jpg"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
    threshold = 0.1  # ここで閾値を指定
    if os.path.isdir(img_path):
        folder_main(img_path, threshold)
    else:
        main(img_path, threshold)


In [ ]:
# 注目点をblurする（反転あり）
import os
import time
import numpy as np
import cv2
import torch
import torch.nn.functional as F
from google.colab.patches import cv2_imshow
from deep_utils import Box, split_extension
import gc

# パラメータ
model_path = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"
img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/スマホ_serial/99.jpg"
output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
img_size = 640
target_layer = 'model_23_cv3_act'
method = 'gradcam'
device = 'cpu'
names = ["infection","normal","non-infection","scar","tumor","deposit","APAC","lens opacity","bullous"]

def get_res_img(bbox, masks, res_img):
    for mask in masks:
        mask = mask.squeeze().mul(255).add_(0.5).clamp_(0, 255).detach().cpu().numpy().astype(np.uint8)
        heatmap = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
        n_heatmat = (Box.fill_outer_box(heatmap, bbox) / 255).astype(np.float32)
        res_img = res_img / 255
        res_img = cv2.add(res_img, n_heatmat)
        res_img = (res_img / res_img.max())
    return res_img, n_heatmat

def put_text_box(bbox, cls_name, res_img):
    x1, y1, x2, y2 = bbox
    cv2.imwrite('temp.jpg', (res_img * 255).astype(np.uint8))
    res_img = cv2.imread('temp.jpg')
    res_img = Box.put_box(res_img, bbox)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2.0
    color = (0, 255, 0)
    thickness = 2

    text_size, _ = cv2.getTextSize(cls_name, font, font_scale, thickness)
    text_x = 10
    text_y = text_size[1] + 10

    cv2.putText(res_img, cls_name, (text_x, text_y), font, font_scale, color, thickness)

    return res_img

def find_yolo_layer(model, layer_name):
    hierarchy = layer_name.split('_')
    target_layer = model.model._modules[hierarchy[0]]

    for h in hierarchy[1:]:
        target_layer = target_layer._modules[h]
    return target_layer

class YOLOV5GradCAM:
    def __init__(self, model, layer_name, img_size=(640, 640)):
        self.model = model
        self.gradients = dict()
        self.activations = dict()
        self.cls_names = []

        def backward_hook(module, grad_input, grad_output):
            self.gradients['value'] = grad_output[0]
            return None

        def forward_hook(module, input, output):
            self.activations['value'] = output
            return None

        target_layer = find_yolo_layer(self.model, layer_name)
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

        device = 'cuda' if next(self.model.model.parameters()).is_cuda else 'cpu'
        self.model(torch.zeros(1, 3, *img_size, device=device))

    def forward(self, input_img, class_idx=True):
        saliency_maps = []
        b, c, h, w = input_img.size()
        preds, logits = self.model(input_img)
        _, top1_idx = torch.topk(logits[0], k=1)

        if top1_idx.numel() > 0:
            preds[1][0] = top1_idx.tolist()[0]
            preds[2][0] = [names[i] for i in preds[1][0]]
            self.cls_names = preds[2][0]
        else:
            self.cls_names = []

        cls, cls_name = preds[1][0][0], preds[2][0][0]
        if class_idx:
            score = logits[0][0][cls]
        else:
            score = logits[0][0].max()
        self.model.zero_grad()
        score.backward(retain_graph=True)
        gradients = self.gradients['value']
        activations = self.activations['value']
        b, k, u, v = gradients.size()
        alpha = gradients.view(b, k, -1).mean(2)
        weights = alpha.view(b, k, 1, 1)
        saliency_map = (weights * activations).sum(1, keepdim=True)
        saliency_map = F.relu(saliency_map)
        saliency_map = F.interpolate(saliency_map, size=(h, w), mode='bilinear', align_corners=False)
        saliency_map_min, saliency_map_max = saliency_map.min(), saliency_map.max()
        saliency_map = (saliency_map - saliency_map_min).div(saliency_map_max - saliency_map_min).data
        saliency_maps.append(saliency_map)

        return saliency_maps, logits, preds, self.cls_names

    def __call__(self, input_img):
        return self.forward(input_img)

def main(img_path, threshold=0.5):
    input_size = (img_size, img_size)
    img = cv2.imread(img_path)
    model = YOLOV5TorchObjectDetector(model_path, device, img_size=input_size, names=names)
    torch_img = model.preprocessing(img[..., ::-1])
    if method == 'gradcam':
        saliency_method = YOLOV5GradCAM(model=model, layer_name=target_layer, img_size=input_size)
    masks, logits, [boxes, _, _, _], cls_names = saliency_method(torch_img)
    result = torch_img.squeeze(0).mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).detach().cpu().numpy()
    result = result[..., ::-1]

    if masks:
        mask = masks[0].squeeze().cpu().numpy()
        if mask.max() >= threshold:
            binary_mask = np.where(mask >= threshold, 1, 0).astype(np.uint8)
            black_mask = np.where(mask >= threshold, 0, 1).astype(np.uint8)
            binary_mask = cv2.resize(binary_mask, (result.shape[1], result.shape[0]))
            black_mask = cv2.resize(black_mask, (result.shape[1], result.shape[0]))

            # ぼかし効果を適用
            blurred_result = cv2.GaussianBlur(result, (91, 91), 0)
            masked_res_img = result * binary_mask[..., np.newaxis] + blurred_result * (1 - binary_mask[..., np.newaxis])
            masked_res_img = (masked_res_img / masked_res_img.max()) * 255

            black_blurred_result = cv2.GaussianBlur(result, (91, 91), 0)
            black_masked_res_img = result * black_mask[..., np.newaxis] + black_blurred_result * (1 - black_mask[..., np.newaxis])
            black_masked_res_img = (black_masked_res_img / black_masked_res_img.max()) * 255

            cv2_imshow(masked_res_img.astype(np.uint8))
            cv2_imshow(black_masked_res_img.astype(np.uint8))

if __name__ == '__main__':
    target_layer = 'model_23_cv3_conv'
    img_path = "/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問/フォトスリット_serial/41.jpg"
    output_dir = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/GradCam_img_slit'
    threshold = 0.3  # ここで閾値を指定
    if os.path.isdir(img_path):
        folder_main(img_path, threshold)
    else:
        main(img_path, threshold)


#**Analyze results**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Load the CSV files
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# Define class names
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# Create confusion matrices for Slit and Sumaho datasets
conf_matrix_slit = pd.crosstab(df_slit['class_num'], df_slit['top1'])
conf_matrix_sumaho = pd.crosstab(df_sumaho['class_num'], df_sumaho['top1'])

# Rename the index and columns for better readability
conf_matrix_slit.index = class_names
conf_matrix_slit.columns = class_names
conf_matrix_sumaho.index = class_names
conf_matrix_sumaho.columns = class_names

# Plotting confusion matrices using seaborn
plt.figure(figsize=(12, 6))

# Slit Lamp Data Confusion Matrix
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix_slit, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix for Slit Lamp Data')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')

# Smartphone Data Confusion Matrix
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_sumaho, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix for Smartphone Data')
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')

plt.tight_layout()
plt.show()


In [ ]:
# Calculate accuracy for Slit and Sumaho datasets
accuracy_slit = (conf_matrix_slit.values.diagonal().sum() / conf_matrix_slit.values.sum()) * 100
accuracy_sumaho = (conf_matrix_sumaho.values.diagonal().sum() / conf_matrix_sumaho.values.sum()) * 100

accuracy_slit, accuracy_sumaho


(85.08771929824562, 77.63157894736842)

In [ ]:
import pandas as pd

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'


# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# AOI_0.5の平均と標準偏差を計算
mean_sd_aoi_slit = df_slit.groupby('top1')['AOI_0.5'].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)
mean_sd_aoi_sumaho = df_sumaho.groupby('top1')['AOI_0.5'].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)

# インデックスにクラス名を設定
mean_sd_aoi_slit.index = class_names
mean_sd_aoi_sumaho.index = class_names

# 結果を表示
print("Mean ± SD AOI for Slit Lamp Data")
print(mean_sd_aoi_slit)
print("\nMean ± SD AOI for Smartphone Data")
print(mean_sd_aoi_sumaho)


Mean ± SD AOI for Slit Lamp Data
                   mean       std
infection      0.086643  0.037705
normal         0.050446  0.016585
non-infection  0.271836  0.145000
scar           0.205724  0.104553
tumor          0.195092  0.097643
deposit        0.072846  0.041480
APAC           0.068622  0.036099
lens opacity   0.137005  0.033883
bullous        0.126359  0.041344

Mean ± SD AOI for Smartphone Data
                   mean       std
infection      0.145351  0.055593
normal         0.114119  0.097931
non-infection  0.421066  0.225339
scar           0.358542  0.173204
tumor          0.247933  0.077559
deposit        0.172425  0.121422
APAC           0.108903  0.041468
lens opacity   0.241247  0.129700
bullous        0.191174  0.048210


In [ ]:
import pandas as pd

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# 列名のリストを作成
columns_of_interest = ['AOI_0.5_layer17', 'AOI_0.5_layer20', 'AOI_0.5_layer23']

# AOIの平均と標準偏差を計算
mean_sd_aoi_slit = df_slit.groupby('top1')[columns_of_interest].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)
mean_sd_aoi_sumaho = df_sumaho.groupby('top1')[columns_of_interest].agg(['mean', 'std']).reindex(range(len(class_names))).fillna(0)

# インデックスにクラス名を設定
mean_sd_aoi_slit.index = class_names
mean_sd_aoi_sumaho.index = class_names

# 結果を表示
print("Mean ± SD AOI for Slit Lamp Data")
print(mean_sd_aoi_slit)
print("\nMean ± SD AOI for Smartphone Data")
print(mean_sd_aoi_sumaho)


Mean ± SD AOI for Slit Lamp Data
              AOI_0.5_layer17           AOI_0.5_layer20            \
                         mean       std            mean       std   
infection            0.085941  0.068294        0.008331  0.045342   
normal               0.178015  0.130984        0.008429  0.033492   
non-infection        0.138709  0.141636        0.003888  0.008722   
scar                 0.164444  0.123306        0.010008  0.034294   
tumor                0.140423  0.102705        0.000232  0.001099   
deposit              0.176102  0.104585        0.001737  0.009827   
APAC                 0.129491  0.132233        0.000000  0.000000   
lens opacity         0.114988  0.097340        0.061234  0.161342   
bullous              0.128982  0.064899        0.009988  0.039860   

              AOI_0.5_layer23            
                         mean       std  
infection            0.086824  0.038336  
normal               0.050446  0.016585  
non-infection        0.271836  0.145000

##**Layerごとに解析**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

####################
layer = 17
#17, 20, 23
####################
layer_name = f"AOI_0.5_layer{layer}"

# ファイルパスを指定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形してSeabornでプロットできるようにする
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 必要なカラムだけを選択し、欠損値を削除して結合
df_combined = pd.concat([
    df_slit[['class_name', layer_name, 'Type']].dropna(),
    df_sumaho[['class_name', layer_name, 'Type']].dropna()
]).reset_index(drop=True)

# グラフのサイズを設定
plt.figure(figsize=(18, 8))

# Seabornのboxplotを使用してプロット
sns.boxplot(x='class_name', y=layer_name, hue='Type', data=df_combined)

# ラベルとタイトルを設定
plt.xlabel('Class')
plt.ylabel(layer_name)
plt.title('Box Plot of AOI_0.5 for Slit Lamp and Smartphone Data')

# グラフを表示
plt.tight_layout()
plt.show()


In [ ]:
# 各クラスごとに対応のあるt検定を行う
results = []
for class_name in class_names:
    slit_data = df_slit[df_slit['class_name'] == class_name][layer_name].dropna().reset_index(drop=True)
    sumaho_data = df_sumaho[df_sumaho['class_name'] == class_name][layer_name].dropna().reset_index(drop=True)

    # 対応のあるデータを取るため、最小の長さに合わせる
    min_length = min(len(slit_data), len(sumaho_data))
    slit_data = slit_data[:min_length]
    sumaho_data = sumaho_data[:min_length]

    t_stat, p_value = stats.ttest_rel(slit_data, sumaho_data)

    # スリットランプデータの統計値
    slit_mean = slit_data.mean()
    slit_std = slit_data.std()

    # スマートフォンデータの統計値
    sumaho_mean = sumaho_data.mean()
    sumaho_std = sumaho_data.std()

    results.append({
        'class_name': class_name,
        't_stat': t_stat,
        'p_value': p_value,
        'slit_mean': slit_mean,
        'slit_std': slit_std,
        'sumaho_mean': sumaho_mean,
        'sumaho_std': sumaho_std
    })

# 全クラスまとめた対応のあるt検定
all_slit_data = df_slit[layer_name].dropna().reset_index(drop=True)
all_sumaho_data = df_sumaho[layer_name].dropna().reset_index(drop=True)

# 対応のあるデータを取るため、最小の長さに合わせる
min_length_all = min(len(all_slit_data), len(all_sumaho_data))
all_slit_data = all_slit_data[:min_length_all]
all_sumaho_data = all_sumaho_data[:min_length_all]

t_stat_all, p_value_all = stats.ttest_rel(all_slit_data, all_sumaho_data)

# 全クラスまとめた統計値
all_slit_mean = all_slit_data.mean()
all_slit_std = all_slit_data.std()
all_sumaho_mean = all_sumaho_data.mean()
all_sumaho_std = all_sumaho_data.std()

# 結果を追加
results.append({
    'class_name': 'All Classes',
    't_stat': t_stat_all,
    'p_value': p_value_all,
    'slit_mean': all_slit_mean,
    'slit_std': all_slit_std,
    'sumaho_mean': all_sumaho_mean,
    'sumaho_std': all_sumaho_std
})

# 結果をデータフレームに変換して表示
df_results = pd.DataFrame(results)
df_results['p_value'] = df_results['p_value'].map(lambda x: f'{x:.3f}')

df_results

,class_name,t_stat,p_value,slit_mean,slit_std,sumaho_mean,sumaho_std
0,infection,0.538977,0.594,0.084249,0.067905,0.074083,0.089991
1,normal,4.106644,0.000,0.181992,0.127307,0.073866,0.110189
2,non-infection,-0.342441,0.738,0.138709,0.141636,0.169776,0.242774
3,scar,3.530070,0.001,0.164444,0.123306,0.082617,0.081827
4,tumor,-3.784626,0.001,0.140423,0.102705,0.287111,0.175874
5,deposit,0.776070,0.444,0.176102,0.104585,0.154578,0.121643
6,APAC,-0.077923,0.941,0.142654,0.159529,0.148437,0.114314
7,lens opacity,0.898269,0.382,0.114988,0.097340,0.084550,0.125398
8,bullous,-1.065212,0.299,0.129807,0.067976,0.163363,0.145041
9,All Classes,0.563571,0.574,0.143497,0.109576,0.136555,0.146311


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# ANOVAを実行
anova_result = stats.f_oneway(
    df_combined[df_combined['class_name'] == 'infection'][layer_name],
    df_combined[df_combined['class_name'] == 'normal'][layer_name],
    df_combined[df_combined['class_name'] == 'non-infection'][layer_name],
    df_combined[df_combined['class_name'] == 'scar'][layer_name],
    df_combined[df_combined['class_name'] == 'tumor'][layer_name],
    df_combined[df_combined['class_name'] == 'deposit'][layer_name],
    df_combined[df_combined['class_name'] == 'APAC'][layer_name],
    df_combined[df_combined['class_name'] == 'lens opacity'][layer_name],
    df_combined[df_combined['class_name'] == 'bullous'][layer_name]
)

print(f"ANOVA result: F={anova_result.statistic}, p={anova_result.pvalue}")

# 事後検定（TukeyのHSD検定）を実行
tukey_result = pairwise_tukeyhsd(df_combined[layer_name], df_combined['class_name'])

# 結果を表示
print(tukey_result)

ANOVA result: F=5.279073563921913, p=2.4769766159597473e-06
       Multiple Comparison of Means - Tukey HSD, FWER=0.05        
    group1        group2    meandiff p-adj   lower   upper  reject
------------------------------------------------------------------
         APAC       bullous   0.0077    1.0  -0.107  0.1223  False
         APAC       deposit   0.0318 0.9933 -0.0792  0.1429  False
         APAC     infection  -0.0569 0.8163 -0.1691  0.0554  False
         APAC  lens opacity  -0.0332 0.9934 -0.1495   0.083  False
         APAC non-infection   0.0071    1.0 -0.1154  0.1297  False
         APAC        normal  -0.0029    1.0 -0.1154  0.1096  False
         APAC          scar  -0.0116    1.0 -0.1213   0.098  False
         APAC         tumor   0.0767 0.4546 -0.0356   0.189  False
      bullous       deposit   0.0242 0.9838 -0.0496  0.0979  False
      bullous     infection  -0.0645 0.1658 -0.1402  0.0111  False
      bullous  lens opacity  -0.0409 0.8223 -0.1223  0.0405  False
  

###**ANOVA_heatmap**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# ファイルパスを設定
file_path1 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_slit_total_updated.csv'
file_path2 = '/gdrive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_GradCAM/maehara_sumaho_total_updated.csv'

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 必要なカラムだけを選択し、欠損値を削除して結合
df_combined = pd.concat([
    df_slit[['class_name', layer_name, 'Type']].dropna(),
    df_sumaho[['class_name', layer_name, 'Type']].dropna()
]).reset_index(drop=True)

# 事後検定（TukeyのHSD検定）を実行
tukey_result = pairwise_tukeyhsd(endog=df_combined[layer_name], groups=df_combined['class_name'])

# 結果をDataFrameに変換
tukey_result_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

# マトリックス図を作成するためにクラス名をインデックスとして使用
matrix = pd.DataFrame(np.zeros((len(class_names), len(class_names))), index=class_names, columns=class_names)

# マトリックスにp値を入力
for i in range(len(tukey_result_df)):
    group1 = tukey_result_df.loc[i, 'group1']
    group2 = tukey_result_df.loc[i, 'group2']
    p_value = float(tukey_result_df.loc[i, 'p-adj'])
    matrix.loc[group1, group2] = p_value
    matrix.loc[group2, group1] = p_value

# 同じクラス間のセルをNaNに設定
np.fill_diagonal(matrix.values, np.nan)

# ヒートマップを描画
plt.figure(figsize=(12, 10))
sns.heatmap(matrix, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'p-value'})
plt.title('Pairwise Comparison P-Values (Tukey HSD)')
plt.show()


###**Tukey_sumaho/slit別**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

def perform_anova_and_tukey(df, title):
    # 欠損値を削除
    df = df.dropna(subset=[layer_name])

    # ANOVAを実行
    anova_result = stats.f_oneway(
        df[df['class_name'] == 'infection'][layer_name],
        df[df['class_name'] == 'normal'][layer_name],
        df[df['class_name'] == 'non-infection'][layer_name],
        df[df['class_name'] == 'scar'][layer_name],
        df[df['class_name'] == 'tumor'][layer_name],
        df[df['class_name'] == 'deposit'][layer_name],
        df[df['class_name'] == 'APAC'][layer_name],
        df[df['class_name'] == 'lens opacity'][layer_name],
        df[df['class_name'] == 'bullous'][layer_name]
    )

    print(f"{title} ANOVA result: F={anova_result.statistic}, p={anova_result.pvalue}")

    # 事後検定（TukeyのHSD検定）を実行
    tukey_result = pairwise_tukeyhsd(endog=df[layer_name], groups=df['class_name'])

    # 結果をDataFrameに変換
    tukey_result_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])

    # マトリックス図を作成するためにクラス名をインデックスとして使用
    matrix = pd.DataFrame(np.zeros((len(class_names), len(class_names))), index=class_names, columns=class_names)

    # マトリックスにp値を入力
    for i in range(len(tukey_result_df)):
        group1 = tukey_result_df.loc[i, 'group1']
        group2 = tukey_result_df.loc[i, 'group2']
        p_value = float(tukey_result_df.loc[i, 'p-adj'])
        matrix.loc[group1, group2] = p_value
        matrix.loc[group2, group1] = p_value

    # 同じクラス間のセルをNaNに設定
    np.fill_diagonal(matrix.values, np.nan)

    # ヒートマップを描画
    plt.figure(figsize=(12, 10))
    sns.heatmap(matrix, annot=True, fmt=".3f", cmap="coolwarm", cbar_kws={'label': 'p-value'}, mask=matrix.isnull())
    plt.title(f'Pairwise Comparison P-Values (Tukey HSD) - {title}')
    plt.show()

# スリットランプデータの解析
perform_anova_and_tukey(df_slit, "Slit Lamp Data")

# スマートフォンデータの解析
perform_anova_and_tukey(df_sumaho, "Smartphone Data")

In [ ]:
import pandas as pd
import scipy.stats as stats

# CSVファイルを読み込む
df_slit = pd.read_csv(file_path1)
df_sumaho = pd.read_csv(file_path2)

# クラス名を定義
class_names = ["infection", "normal", "non-infection", "scar", "tumor", "deposit", "APAC", "lens opacity", "bullous"]

# クラス名を数字に対応させる
df_slit['class_name'] = df_slit['top1'].map(dict(enumerate(class_names)))
df_sumaho['class_name'] = df_sumaho['top1'].map(dict(enumerate(class_names)))

# データフレームを整形
df_slit['Type'] = 'Slit'
df_sumaho['Type'] = 'Sumaho'

# 両方のデータを結合
df_combined = pd.concat([df_slit, df_sumaho]).reset_index(drop=True)

# "scar" + "non-infection" グループとその他のクラスに分類
df_combined['group'] = df_combined['class_name'].apply(lambda x: 'scar + non-infection' if x in ['scar', 'non-infection'] else 'others')

# グループごとにデータを抽出
scar_non_infection_group = df_combined[df_combined['group'] == 'scar + non-infection'][layer_name].dropna()
others_group = df_combined[df_combined['group'] == 'others'][layer_name].dropna()

# t検定を実行
t_stat, p_value = stats.ttest_ind(scar_non_infection_group, others_group)

# 結果を表示
print(f"t-statistic: {t_stat}, p-value: {p_value}")

# 統計値を計算
scar_non_infection_mean = scar_non_infection_group.mean()
scar_non_infection_std = scar_non_infection_group.std()
others_mean = others_group.mean()
others_std = others_group.std()

print(f"scar + non-infection mean: {scar_non_infection_mean}, std: {scar_non_infection_std}")
print(f"others mean: {others_mean}, std: {others_std}")


t-statistic: -0.8384780058198069, p-value: 0.40221043504210663
scar + non-infection mean: 0.13079727181747405, std: 0.13409564357929413
others mean: 0.14286203163555233, std: 0.1274883973680255
